#Setup

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar

!pip3 install atari-py

!python -m atari_py.import_roms ROMS

!pip install "gym[atari]" "gym[accept-rom-license]" atari_py

!pip install -U "ray[rllib]==1.11.0"

# Visualizer setup
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1


In [ ]:
!pip install -U "ray[rllib]==1.11.0"

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import gym
import torch

import numpy as np
from collections import namedtuple
import random, math
import matplotlib.pyplot as plt
from ray.rllib.models import ModelCatalog

from ray.rllib.env.env_context import EnvContext
import time
import os

from gym.spaces import Discrete, Box
from ray import tune
import ray
import ray.rllib.agents.dqn as dqn

In [5]:
# Visualizer setup
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1


# Code for visualization

In [7]:
# How to get gym AI visuals in colab
# https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb#scrollTo=T9RpF49oOsZj

from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Test env with random actions


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

game =  'Breakout-ram-v0' 
env =  wrap_env(gym.make(game))

print("game: ", game)
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n

print("States size:", num_states, "Actions size: ",  num_actions)

device:  cuda:0
game:  Breakout-ram-v0
States size: 128 Actions size:  4


In [9]:
done = False

env.reset()

total_reward = 0 
while not done:
    env.render()
    state, reward, done, _ = env.step(env.action_space.sample()) # take a random action
    total_reward += reward

env.close()
show_video()

# Set config based on best hyper param 



In [ ]:
ray.shutdown()
ray.init(num_cpus= 2, num_gpus=1)

{'address': '172.28.0.2:61908',
 'gcs_address': '172.28.0.2:61908',
 'metrics_export_port': 48233,
 'node_id': '41376145bfc1a60cb54fa1f3aed1d94eec824c2cef3ef1a5c1e3c760',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2022-04-22_20-14-10_018819_72/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2022-04-22_20-14-10_018819_72/sockets/raylet',
 'redis_address': None,
 'session_dir': '/tmp/ray/session_2022-04-22_20-14-10_018819_72',
 'webui_url': None}

In [10]:
config = dqn.DEFAULT_CONFIG.copy()

config["dueling"] = True
config["double_q"] = False
config["noisy"] = True
config["prioritized_replay"] = True


#evluation config 
config["evaluation_interval"] =  10
config["evaluation_config"] = {"explore": False} 

config["env"] = 'Breakout-ram-v0' 

config["model"] = {"fcnet_hiddens": [128],
                  "fcnet_activation": 'relu',
                   }

config["num_gpus"] = 1
model_train_config = config

In [ ]:
analysis = tune.run("DQN",
                    stop={"episode_reward_mean": 5},
                    config = model_train_config,
                    local_dir = "/content/drive/MyDrive/INM707/breakout_v1",
                    checkpoint_freq = 10
                    )

#print("Best hyperparameters found were: ", analysis.best_config)

(DQNTrainer pid=1651) 2022-04-22 20:58:53,972	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(DQNTrainer pid=1651) 2022-04-22 20:58:53,973	INFO simple_q.py:155 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(DQNTrainer pid=1651) 2022-04-22 20:58:53,973	INFO trainer.py:781 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(DQNTrainer pid=1651) 2022-04-22 20:58:59,060	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!


Trial name,status,loc
DQN_Breakout-ram-v0_02894_00000,RUNNING,172.28.0.2:1651


(DQNTrainer pid=1651) 2022-04-22 20:59:02,762	WARNING util.py:55 -- Install gputil for GPU system monitoring.


Trial name,status,loc
DQN_Breakout-ram-v0_02894_00000,RUNNING,172.28.0.2:1651


Result for DQN_Breakout-ram-v0_02894_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2022-04-22_20-59-07
  done: false
  episode_len_mean: 259.0
  episode_media: {}
  episode_reward_max: 3.0
  episode_reward_mean: 1.3333333333333333
  episode_reward_min: 0.0
  episodes_this_iter: 3
  episodes_total: 3
  experiment_id: b6c359f203ae460e8406d0db20984161
  hostname: e4fed5a56747
  info:
    last_target_update_ts: 1000
    learner:
      default_policy:
        learner_stats:
          cur_lr: 0.0005000000237487257
          max_q: 0.6463987827301025
          mean_q: 0.07603371888399124
          mean_td_error: -0.4412388503551483
          min_q: -0.8483760356903076
          model: {}
        td_error:
        - -0.419186532497406
        - -0.1831558346748352
        - -0.10791763663291931
        - -0.1420905888080597
        - -0.4121747314929962
        - -0.9172872304916382
        - -0.07762652635574341
        - -1.1191010475158691
        - -0.9250389337539673
 

...

Result for DQN_Breakout-ram-v0_02894_00000:
  agent_timesteps_total: 256000
  custom_metrics: {}
  date: 2022-04-22_21-42-18
  done: false
  episode_len_mean: 426.39
  episode_media: {}
  episode_reward_max: 13.0
  episode_reward_mean: 4.92
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 671
  experiment_id: b6c359f203ae460e8406d0db20984161
  hostname: e4fed5a56747
  info:
    last_target_update_ts: 255520
    learner:
      default_policy:
        learner_stats:
          cur_lr: 0.0005000000237487257
          max_q: 2.399876356124878
          mean_q: 1.6494140625
          mean_td_error: 0.04019375145435333
          min_q: 0.727883517742157
          model: {}
        td_error:
        - 0.049956679344177246
        - 0.1435391902923584
        - 0.04551684856414795
        - 0.024008631706237793
        - 0.017469286918640137
        - -0.002011537551879883
        - 0.02717411518096924
        - 0.09974038600921631
        - 0.06949400901794434
        - 0.00

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_Breakout-ram-v0_02894_00000,RUNNING,172.28.0.2:1651,256,2585.92,256000,4.92,13,0,426.39


Result for DQN_Breakout-ram-v0_02894_00000:
  agent_timesteps_total: 257000
  custom_metrics: {}
  date: 2022-04-22_21-42-25
  done: true
  episode_len_mean: 432.88
  episode_media: {}
  episode_reward_max: 13.0
  episode_reward_mean: 5.03
  episode_reward_min: 0.0
  episodes_this_iter: 2
  episodes_total: 673
  experiment_id: b6c359f203ae460e8406d0db20984161
  hostname: e4fed5a56747
  info:
    last_target_update_ts: 256528
    learner:
      default_policy:
        learner_stats:
          cur_lr: 0.0005000000237487257
          max_q: 2.3730719089508057
          mean_q: 1.5923550128936768
          mean_td_error: 0.0023896507918834686
          min_q: 0.9900310635566711
          model: {}
        td_error:
        - 0.010199308395385742
        - 0.06239354610443115
        - 0.03838825225830078
        - 0.047747135162353516
        - 0.025039076805114746
        - 0.037882447242736816
        - 0.09353387355804443
        - 0.007621645927429199
        - 0.12205779552459717
    

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_Breakout-ram-v0_02894_00000,TERMINATED,172.28.0.2:1651,257,2592.62,257000,5.03,13,0,432.88


2022-04-22 21:42:26,315	INFO tune.py:639 -- Total run time: 2616.24 seconds (2615.51 seconds for the tuning loop).


In [ ]:
df = analysis.dataframe(metric="episode_reward_mean", mode="max")

In [ ]:
df.head()

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,config/synchronize_filters,config/target_network_update_freq,config/tf_session_args,config/timesteps_per_iteration,config/train_batch_size,config/training_intensity,config/v_max,config/v_min,config/worker_side_prioritization,logdir
0,13.0,0.0,5.03,432.88,2,0,257000,32,257000,True,...,True,500,"{'allow_soft_placement': True, 'device_count':...",1000,32,None,10.0,-10.0,False,/content/drive/MyDrive/INM707/breakout_v1/DQN/...


#Restore Agent

In [11]:
checkpoint_file = '/content/drive/MyDrive/INM707/breakout_v1/DQN/DQN_Breakout-ram-v0_02894_00000_0_2022-04-22_20-58-50/checkpoint_000250/checkpoint-250'
agent = dqn.DQNTrainer(config=config)
agent.restore(checkpoint_file)

2022-04-24 12:09:16,164	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
Install gputil for GPU system monitoring.


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


2022-04-24 12:09:25,521	INFO trainable.py:496 -- Restored on 172.28.0.2 from checkpoint: /content/drive/MyDrive/INM707/breakout_v1/DQN/DQN_Breakout-ram-v0_02894_00000_0_2022-04-22_20-58-50/checkpoint_000250/checkpoint-250
2022-04-24 12:09:25,523	INFO trainable.py:503 -- Current state after restoring: {'_iteration': 250, '_timesteps_total': 8000, '_time_total': 2546.3796491622925, '_episodes_total': 659}


In [29]:
import gym
done = False

env = wrap_env(gym.make('Breakout-ram-v0'))
state = env.reset()
total_reward = 0 

while not done:
    action = agent.compute_single_action(state)

    state, reward, done, _ = env.step(action) 
    total_reward += reward

    #env.render()


env.close()
show_video()

In [30]:
print(total_reward)

5.0
